<a href="https://colab.research.google.com/github/wakepon21/Masa/blob/Signate2/benchmark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ライブラリのインポート
import pandas as pd
import lightgbm as lgb
import warnings
import gc
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn import metrics
warnings.simplefilter('ignore')

In [ ]:
#train,test,submit_sampleのみっつがそろっているか確認
!ls

sample_data


In [ ]:
#そろっていなかったら選択
from google.colab import files
train_up = files.upload()

Saving sample_submit.csv to sample_submit.csv
Saving test.csv to test.csv
Saving train.csv to train.csv


In [53]:
submit_df = pd.read_csv("sample_submit.csv",names=("A","B"))
test_df = pd.read_csv("test.csv")
train_df = pd.read_csv("train.csv")

# データの量の確認
train_df.shape,test_df.shape,submit_df.shape

((850, 11), (350, 10), (350, 2))

In [54]:
# 訓練データ、テストデータがわかるようにダミーの目的変数を代入
test_df['disease']=-999

In [55]:
# 訓練データ、テストデータを結合
all_df = pd.concat([train_df,test_df])
del train_df,test_df
gc.collect()

647

In [56]:
# カテゴリカラムの前処理
categorical_features = ["Gender"]
for col in categorical_features:
    lbl = preprocessing.LabelEncoder()
    lbl.fit(all_df[col])
    lbl.transform(all_df[col])
    all_df[col]=lbl.transform(all_df[col])

In [57]:
# 訓練データ、テストデータの分割
train_df = all_df[all_df['disease']!=-999]
test_df = all_df[all_df['disease']==-999]

In [58]:
y_train_origin = train_df['disease']
X_train_origin = train_df.drop(['disease'], axis=1)
X_test_origin = test_df.drop(['disease'], axis=1)

In [59]:
output_df = pd.DataFrame()
for i in range(100):
    #訓練データからテストデータを分割
    X_train , X_valid , y_train , y_valid = train_test_split(X_train_origin,y_train_origin,test_size = 0.3 , random_state = i)
    #使用モデルはLGB（パラメーターチューニングなし）
    lgb_train = lgb.Dataset(X_train,y_train,categorical_feature = categorical_features)
    lgb_eval = lgb.Dataset(X_valid , y_valid , reference = lgb_train , categorical_feature = categorical_features)
    params = {
        "objective":"binary"
    }

    model = lgb.train(
        params,lgb_train,
        valid_sets=[lgb_train,lgb_eval],
        verbose_eval = 10,
        num_boost_round = 1000,
        early_stopping_rounds=10
    )

    y_pred = model.predict(X_test_origin,num_iteration=model.best_iteration)
    output_df[i] = y_pred



Training until validation scores don't improve for 10 rounds.
[10]	training's binary_logloss: 0.356897	valid_1's binary_logloss: 0.386781
[20]	training's binary_logloss: 0.241681	valid_1's binary_logloss: 0.299914
[30]	training's binary_logloss: 0.171527	valid_1's binary_logloss: 0.261943
[40]	training's binary_logloss: 0.125409	valid_1's binary_logloss: 0.244551
[50]	training's binary_logloss: 0.0914845	valid_1's binary_logloss: 0.23815
[60]	training's binary_logloss: 0.0662456	valid_1's binary_logloss: 0.237305
Early stopping, best iteration is:
[59]	training's binary_logloss: 0.0685872	valid_1's binary_logloss: 0.235164
Training until validation scores don't improve for 10 rounds.
[10]	training's binary_logloss: 0.349835	valid_1's binary_logloss: 0.413977
[20]	training's binary_logloss: 0.226958	valid_1's binary_logloss: 0.324278
[30]	training's binary_logloss: 0.162315	valid_1's binary_logloss: 0.307425
[40]	training's binary_logloss: 0.11866	valid_1's binary_logloss: 0.299668
Earl

In [64]:
from sklearn.metrics import accuracy_score
percentage_LGB=0.3
y_pred_fin = output_df.mean(axis = 1)
#y_sub_LGB = sum(y_preds) / len(y_preds)
y_sub = (y_pred > percentage_LGB).astype(int)

submit_df["B"]=y_sub
submit_df.to_csv('submit_test.csv', index=False, header=False)
submit_df.head()



,A,B
0,0,0
1,1,1
2,2,0
3,3,0
4,4,0


In [70]:
output_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99
0,0.004702,0.021260,0.008639,0.015738,0.009862,0.023914,0.032604,0.011886,0.017469,0.017155,0.006398,0.012180,0.009505,0.040283,0.020470,0.028410,0.023151,0.044290,0.015008,0.029325,0.016248,0.026531,0.028259,0.012811,0.013008,0.047603,0.008486,0.016892,0.031181,0.023770,0.012532,0.017247,0.029692,0.014029,0.011198,0.010578,0.036858,0.021084,0.016156,0.017488,...,0.008881,0.024527,0.014314,0.015825,0.007210,0.025841,0.035333,0.019956,0.014118,0.014305,0.020606,0.012148,0.011889,0.020353,0.010436,0.005342,0.015073,0.007208,0.017555,0.029553,0.043822,0.013829,0.018547,0.027601,0.033380,0.015125,0.002483,0.008481,0.006676,0.038221,0.031353,0.016741,0.036852,0.024032,0.037439,0.007361,0.016260,0.014375,0.037457,0.030835
1,0.467204,0.268642,0.130392,0.221459,0.662869,0.174224,0.366675,0.321188,0.380039,0.138031,0.064753,0.078884,0.158063,0.187518,0.246028,0.153735,0.278369,0.117534,0.197549,0.477500,0.163742,0.354168,0.194013,0.181632,0.082914,0.382064,0.158923,0.076477,0.359082,0.328868,0.161573,0.211758,0.183804,0.225270,0.131161,0.181306,0.641625,0.234535,0.339622,0.195863,...,0.250010,0.406030,0.288199,0.188608,0.446282,0.243214,0.419993,0.185247,0.073408,0.242869,0.289780,0.553350,0.473819,0.221754,0.075858,0.261542,0.222327,0.186152,0.323273,0.217372,0.286934,0.296764,0.211130,0.336977,0.294633,0.298504,0.060025,0.130622,0.047436,0.314228,0.321169,0.311798,0.279093,0.151946,0.304039,0.246636,0.184968,0.219405,0.298643,0.319475
2,0.001993,0.008818,0.004769,0.011481,0.008770,0.021615,0.042929,0.017555,0.011563,0.019316,0.004404,0.008890,0.008754,0.035745,0.018990,0.025691,0.015542,0.026875,0.019010,0.025224,0.009780,0.025220,0.020309,0.012340,0.014860,0.080234,0.012693,0.009082,0.038183,0.014410,0.006542,0.008537,0.022572,0.017589,0.022860,0.009598,0.038316,0.027836,0.016274,0.014387,...,0.010296,0.021315,0.012014,0.017732,0.005233,0.020007,0.044639,0.021422,0.013728,0.016543,0.020551,0.006580,0.012954,0.006984,0.007922,0.004199,0.030611,0.009226,0.025265,0.023639,0.025971,0.014458,0.013246,0.024719,0.043114,0.011896,0.001826,0.012024,0.005625,0.041229,0.028435,0.009715,0.024905,0.020297,0.017200,0.012100,0.019440,0.013148,0.039222,0.020490
3,0.066553,0.057493,0.082297,0.027466,0.085489,0.114019,0.117804,0.093037,0.048445,0.137219,0.042447,0.027276,0.059036,0.088614,0.118063,0.098971,0.087282,0.076192,0.039715,0.060116,0.060552,0.065249,0.066101,0.074695,0.055073,0.142449,0.047002,0.052213,0.162544,0.048822,0.057765,0.061788,0.076252,0.129834,0.108836,0.095637,0.042112,0.060619,0.088194,0.059039,...,0.148272,0.140164,0.061259,0.151654,0.093488,0.053849,0.144793,0.129326,0.038617,0.073177,0.044425,0.021991,0.053031,0.093348,0.128139,0.036508,0.078303,0.019413,0.107766,0.080120,0.048282,0.094239,0.041845,0.087002,0.161701,0.047639,0.076460,0.124201,0.049763,0.060585,0.096182,0.028244,0.152077,0.066801,0.069295,0.060278,0.191969,0.028837,0.063664,0.094514
4,0.010984,0.026194,0.016028,0.028744,0.017275,0.064701,0.026352,0.045628,0.022945,0.046043,0.032279,0.060036,0.021752,0.067665,0.076109,0.041323,0.054668,0.045451,0.081171,0.057877,0.048115,0.042059,0.025430,0.015077,0.041706,0.101688,0.040118,0.027942,0.040227,0.045827,0.045220,0.028805,0.046912,0.028432,0.025471,0.021843,0.094209,0.041455,0.091371,0.042928,...,0.029420,0.033063,0.018725,0.050115,0.017315,0.040963,0.047842,0.027924,0.053845,0.032939,0.027949,0.017926,0.029639,0.031601,0.009536,0.031420,0.030474,0.031170,0.046687,0.030200,0.059149,0.044720,0.044177,0.026469,0.076348,0.023908,0.035031,0.013468,0.018153,0.074423,0.056131,0.038564,0.035467,0.044781,0.037750,0.056847,0.028048,0.012804,0.066408,0.028097
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [65]:
sum(y_sub),len(y_sub)

(175, 350)

In [69]:
files.download('submit_test.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [63]:
67f6u7=ihoioo;gh@0]h

SyntaxError: ignored